In [ ]:
# Ejemplo 2:
# Modelo TROPN, obtención del arbol de alcanzabilidad factible en el tiempo,
# y obtención de ruta óptima para el modelo propuesto por Chen et al. (2019)


In [ ]:
import snakes.plugins
import tropn
snakes.plugins.load(tropn, "snakes.nets", "nets")
from nets import *
from tropn import *
from time import process_time

In [ ]:
n = PetriNet('TROPN-Chen-2019')
n.globals.declare("from tropn import process")

n.add_place(Place('p0',[], bound=0))
n.add_place(Place('p1',[], bound=1))
n.add_place(Place('p2',[], bound=1))
n.add_place(Place('p3',[], bound=1))
n.add_place(Place('p4',[], bound=2))
n.add_place(Place('p5',[], bound=2))
n.add_place(Place('p6',[], bound=2))
n.add_place(Place('p7',[], bound=2))

n.add_transition(Transition('t1', Expression('(x=="X1") and dot=="dot"')))
n.add_transition(Transition('t2', Expression('(x=="X1") and dot=="dot"')))
n.add_transition(Transition('t3', Expression('(x=="X1_1") and dot=="dot"')))
n.add_transition(Transition('t4', Expression('(x=="X1_2") and dot=="dot"')))
n.add_transition(Transition('t5', Expression('(x=="X1") and dot=="dot"')))
n.add_transition(Transition('t6', Expression('(x=="X1") and dot=="dot"')))
n.add_transition(Transition('t7', Expression('(x=="X1_1") and dot=="dot"')))
n.add_transition(Transition('t8', Expression('(x=="X1_2") and dot=="dot"')))
n.add_transition(Transition('t9', Expression('(x=="X2") and dot=="dot"')))
n.add_transition(Transition('t10', Expression('(x=="X2_1") and dot=="dot"')))
n.add_transition(Transition('t11', Expression('(x=="X3") and dot=="dot"')))
n.add_transition(Transition('t12', Expression('(x=="X3_1") and dot=="dot"')))
n.add_transition(Transition('t13', Expression('(x=="X3_2") and dot=="dot"')))
                                                        
for t in ['t1','t5','t13']:
    n.add_input('p1', t, MultiArc([Variable("dot")]))
    n.add_output('p1', t, MultiArc([Expression("dot")]))  

for t in ['t2','t3','t6', 't7', 't9', 't10', 't12']:
    n.add_input('p2', t, MultiArc([Variable("dot")]))
    n.add_output('p2', t, MultiArc([Expression("dot")]))  
    
for t in ['t4','t8','t11']:
    n.add_input('p3', t, MultiArc([Variable("dot")]))
    n.add_output('p3', t, MultiArc([Expression("dot")]))  

n.add_input('p0', 't1', MultiArc([Variable('x')]))
n.add_output('p4', 't1', MultiArc([Expression('process(x)')]))
n.add_input('p0', 't2', MultiArc([Variable('x')]))
n.add_output('p4', 't2', MultiArc([Expression('process(x)')]))
n.add_input('p4', 't3', MultiArc([Variable('x')]))
n.add_output('p5', 't3', MultiArc([Expression('process(x)')]))
n.add_input('p5', 't4', MultiArc([Variable('x')]))
n.add_output('p0', 't4', MultiArc([Expression('x')]))
n.add_input('p0', 't5', MultiArc([Variable('x')]))
n.add_output('p6', 't5', MultiArc([Expression('process(x)')]))
n.add_input('p0', 't6', MultiArc([Variable('x')]))
n.add_output('p6', 't6', MultiArc([Expression('process(x)')]))
n.add_input('p6', 't7', MultiArc([Variable('x')]))
n.add_output('p7', 't7', MultiArc([Expression('process(x)')]))
n.add_input('p7', 't8', MultiArc([Variable('x')]))
n.add_output('p0', 't8', MultiArc([Expression('x')]))
n.add_input('p0', 't9', MultiArc([Variable('x')]))
n.add_output('p5', 't9', MultiArc([Expression('process(x)')]))
n.add_input('p5', 't10', MultiArc([Variable('x')]))
n.add_output('p0', 't10', MultiArc([Expression('x')]))
n.add_input('p0', 't11', MultiArc([Variable('x')]))
n.add_output('p7', 't11', MultiArc([Expression('process(x)')]))
n.add_input('p7', 't12', MultiArc([Variable('x')]))
n.add_output('p6', 't12', MultiArc([Expression('process(x)')]))
n.add_input('p6', 't13', MultiArc([Variable('x')]))
n.add_output('p0', 't13', MultiArc([Expression('x')]))

n.set_marking(Marking({'p0': MultiSet(["X1", "X2","X3"]),
                      'p1': MultiSet(["dot"]),
                      'p2': MultiSet(["dot"]),
                      'p3': MultiSet(["dot"])}))
                
M0 = n.get_marking()
M0

In [ ]:
# Los tiempos de espera se definen como diccionario
# Para cada token, basta señalar los tiempos de espera correspondientes
# en las plazas y transiciones pertinentes.
# Cualquier tiempo de espera que no esté declara explicitamente, será cero
n.delays = {'X1':{'t1':3,'t2':3},
            'X1_1':{'p4':5,'p6':5,'t3':3,'t7':3},
            'X1_2':{'p5':5,'p7':5,'t4':3,'t8':3},

            'X2':{'t9':3},
            'X2_1':{'p5':5,'t10':3},

            'X3':{'t11':3},
            'X3_1':{'p7':5,'t12':3},
            'X3_2':{'p6':5,'t13':3},
        }


In [ ]:
# El procedimiento general de simulación es el siguiente
time_start = process_time()
# 1. Definir el estado inicial a partir de un marcaje inicial, con
#    ningún proceso actualmente en ejecución
m = (Marking({'p0': MultiSet(["X1","X2","X3"]*1),
              'p1': MultiSet(["dot"]),
              'p2': MultiSet(["dot"]),
              'p3': MultiSet(["dot"])}))
s0 = NetState(m,{},{})
set_state(n,s0)
# 2. Obtener el árbol de alcanzabilidad
markings, tree, optimal = time_feasible_reachability_tree(n,s0)
# ....................
time_elapsed = (process_time() - time_start)
print ("%5.3f secs" % (time_elapsed))
print (len(markings), "states found")
print (len(tree), "state transitions")

In [ ]:
# número de estados distintos
print ('estados', len(markings))
print ('pasos', len(tree))
print ('longitud', len(optimal))
print ('tiempo', optimal[-1][5])

In [ ]:
## número de pasos en el árbol de alcanzabilidad factible en el tiempo
len(tree)

In [ ]:
# longitud de la ruta de mínimo tiempo de procesamiento
len(optimal)

In [ ]:
# la estructura de cada paso es:
# step[0]: marcaje inicial
# step[1]: marcaje final
# step[2]: transición disparada
# step[3]: tokens utilizados
# step[4]: tiempo que tardó en ejecutarse el paso
# step[5]: tiempo de procesamiento acumulado

for step in optimal:
    print (step[5])
    print (step[0])
    print (step[1])
    print (step[2], step[3])